In [1]:
import torch
import torch.nn as nn

# Definir el bloque básico de la red neuronal
class BasicBlock(nn.Module):
    expansion = 1 # Factor de expansión de los canales de salida
    def __init__(self, in_channels, out_channels, stride=1): # stride=1: no se reduce el tamaño de la imagen
        super(BasicBlock, self).__init__()
        # primera capa convolucional (3x3) con stride=1 y padding=1 para mantener el tamaño de la imagen
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels) # Normalización por lotes
        self.relu = nn.ReLU(inplace=True) # Función de activación
        # segunda capa convolucional (3x3) con stride=1 y padding=1 para mantener el tamaño de la imagen
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels) # Normalización por lotes

        self.downsample = None # No se reduce el tamaño de la imagen
        # Si se reduce el tamaño de la imagen se aplica una capa convolucional (1x1) con stride=2 y padding=0
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.downsample = nn.Sequential( # Capa secuencial
                # Capa convolucional (1x1) con stride=2 y padding=0
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion) # Normalización por lotes
            )

    def forward(self, x):
        identity = x # Guardar la entrada para la conexión residual

        out = self.conv1(x) # Aplicar la primera capa convolucional
        out = self.bn1(out) # Aplicar la normalización por lotes
        out = self.relu(out) # Aplicar la función de activación

        out = self.conv2(out) # Aplicar la segunda capa convolucional
        out = self.bn2(out) # Aplicar la normalización por lotes

        if self.downsample is not None: # Si se reduce el tamaño de la imagen se aplica la capa secuencial
            identity = self.downsample(x) # Aplicar la capa secuencial para reducir el tamaño de la imagen de entrada

        out += identity # Conexión residual (suma) entre la entrada y la salida de la capa convolucional
        out = self.relu(out) # Aplicar la función de activación
        # La salida de la capa convolucional es la entrada de la siguiente capa convolucional
        return out

In [2]:
# Definir la arquitectura de la red neuronal principal (ResNet)
class Model_frontscratsh(nn.Module):
    def __init__(self, block=BasicBlock, layers=[2, 2, 2, 2], num_classes=12):
        super(Model_frontscratsh, self).__init__()
        self.in_channels = 64 # Número de canales de entrada
        # Capa convolucional inicial (7x7) con stride=2 y padding=3 para reducir el tamaño de la imagen
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64) # Normalización por lotes
        self.relu = nn.ReLU(inplace=True) # Función de activación
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # Capa de maxpooling (3x3) con stride=2 y padding=1
        
        # Capas residuales
        self.layer1 = self._make_layer(block, 64, layers[0]) # Capa residual 1 (64 canales de salida)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2) # Capa residual 2 (128 canales de salida) 
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2) # Capa residual 3 (256 canales de salida)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2) # Capa residual 4 (512 canales de salida)

        # Capa de clasificación
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # Capa de pooling adaptativo (1x1) para reducir el tamaño de la imagen
        # Capa de clasificación (12 clases) con 512 canales de entrada
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    # Función para crear una capa residual (bloque residual)
    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = [] # Lista de capas
        layers.append(block(self.in_channels, out_channels, stride)) # Capa residual
        self.in_channels = out_channels * block.expansion # Actualizar el número de canales de entrada
        for _ in range(1, blocks):
            # Capa residual con el mismo número de canales de entrada y salida
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x) # Aplicar la capa convolucional inicial
        x = self.bn1(x) # Aplicar la normalización por lotes
        x = self.relu(x) # Aplicar la función de activación
        x = self.maxpool(x) # Aplicar la capa de maxpooling

        x = self.layer1(x) # Aplicar la capa residual 1
        x = self.layer2(x) # Aplicar la capa residual 2
        x = self.layer3(x) # Aplicar la capa residual 3
        x = self.layer4(x) # Aplicar la capa residual 4

        x = self.avgpool(x) # Aplicar la capa de pooling adaptativo
        x = torch.flatten(x, 1) # Aplanar la salida de la capa de pooling adaptativo
        x = self.fc(x) # Aplicar la capa de clasificación

        # Salida de la capa de clasificación
        return x

In [3]:
model = Model_frontscratsh()
print(model)

Model_frontscratsh(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inpl